 # Clasificador de gatos y perros con una CNN


## Comenzando

### Prerequisitos

Necesitaremos:
* Tensorflow: Librería de machine learning creada poro google, se instala con *pip install tensorflow*
* Keras:      Framework que simplifica el uso de tensorflow, se instala con *pip install keras*
* pickle:     Nos ayuda a cargar el dataset preprocesado, viene instalado por defecto.
* Opencv:     Nos ayuda a mostrar las fotos en la parte final del código.


Además de estos paquetes, necesitará la base de datos preprocesada que se facilitará

Tambien puede visualizar la base de datos sin procesar en el siguiente link:

https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765


### Comenzamos importando los paquetes que utilizaremos

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle



### Ahora cargaremos la base de datos preprocesada, X correspondiendo a las imagenes desordenadas e y a sus respectivas etiquetas.

In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

### Normalizar los valores genera una mejora considerable en los resultados

In [ ]:
X = X/255.0

### Ahora generaremos nuestro modelo

La estructura básica de una CNN es: 

* Convolución -> Agrupación (pooling) -> Convolución -> Agrupación(Pooling) -> Capa completamente conectada -> Salida

<img src="http://www.mdpi.com/entropy/entropy-19-00242/article_deploy/html/images/entropy-19-00242-g001.png"> 

In [ ]:
#Primero diremos que es secuencial la forma en la que se conectan las capas.
model = Sequential()
#Ahora podemos ir añadiendo o quitando capas.
#Comenzaremos con convolutional layer que será de dos dimensiones ya que es la forma de nuestras imagenes.
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
#Ahora añadimos nuestra función de activación
model.add(Activation('relu'))
# Agrupamos utilizando maxpooling, que simplemente toma el valor máximo de la ventana que analiza .
model.add(MaxPooling2D(pool_size=(2, 2)))

#repetimos otra vez
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Ahora con Flatten convertiremos nuestro mapa de caracterizticas 3D a un vector de 1D.
model.add(Flatten())
#Futuro análisis revela que para este caso en específico no necesitamos Dense layer, mas se mantendrá por metodología.
model.add(Dense(64))
model.add(Dense(1))

#Ahora añadiremos nuestra función de activación, tener un entendimiento de la salida del modelo, nos ayuda a elegir el tipo.
model.add(Activation('sigmoid'))

#Por ultimo compilaremos nuestro modelo, especificando como se moverá nuestra pérdida, que optimizador utilizaremos y que valor mediremos.
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3)

### Para optimizar el modelo se suele utilizar la regla de moverse +- una capa y comprobar su desempeño, buscando indicios de overfiting 

##### En este caso veremos las siguientes combinaciones
* 1-conv-32-nodes-0-dense-1534801383
* 2-conv-32-nodes-0-dense-1534801383
* 3-conv-32-nodes-0-dense-1534801383
* 1-conv-64-nodes-0-dense-1534801383
* 2-conv-64-nodes-0-dense-1534801383
* 3-conv-64-nodes-0-dense-1534801383
* 1-conv-128-nodes-0-dense-1534801383
* 2-conv-128-nodes-0-dense-1534801383
* 3-conv-128-nodes-0-dense-1534801383
* 1-conv-32-nodes-1-dense-1534801383
* 2-conv-32-nodes-1-dense-1534801383
* 3-conv-32-nodes-1-dense-1534801383
* 1-conv-64-nodes-1-dense-1534801383
* 2-conv-64-nodes-1-dense-1534801383
* 3-conv-64-nodes-1-dense-1534801383
* 1-conv-128-nodes-1-dense-1534801383
* 2-conv-128-nodes-1-dense-1534801383
* 3-conv-128-nodes-1-dense-1534801383
* 1-conv-32-nodes-2-dense-1534801383
* 2-conv-32-nodes-2-dense-1534801383
* 3-conv-32-nodes-2-dense-1534801383
* 1-conv-64-nodes-2-dense-1534801383
* 2-conv-64-nodes-2-dense-1534801383
* 3-conv-64-nodes-2-dense-1534801383
* 1-conv-128-nodes-2-dense-1534801383
* 2-conv-128-nodes-2-dense-1534801383
* 3-conv-128-nodes-2-dense-1534801383

<img src="https://pythonprogramming.net/static/images/machine-learning/optimizing-models.png"> 

### Con esto podemos ver el top 10 de modelos
* 3 conv, 64 nodes per layer, 0 dense
* 3 conv, 128 nodes per layer, 0 dense
* 3 conv, 32 nodes per layer, 0 dense
* 3 conv, 32 nodes per layer, 2 dense
* 3 conv, 32 nodes per layer, 1 dense
* 2 conv, 32 nodes per layer, 0 dense
* 2 conv, 64 nodes per layer, 0 dense
* 3 conv, 128 nodes per layer, 1 dense
* 2 conv, 128 nodes per layer, 0 dense
* 2 conv, 32 nodes per layer, 1 dense

### Por último para utilizar este modelo debemos utilizar el siguiente código:

In [2]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.model")

prediction = model.predict([prepare('doggo.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[0.]]
Dog
